# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [1]:
#!pip install -qU ragas==0.2.10

In [3]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [2]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [3]:
!mkdir data

In [4]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31506    0 31506    0     0  47842      0 --:--:-- --:--:-- --:--:-- 47808


In [5]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70532    0 70532    0     0   251k      0 --:--:-- --:--:-- --:--:--  251k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/cantemizyurek/Documents/code/ai/ai-maker-space/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/cantemizyurek/Documents/code/ai/ai-maker-space/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/cantemizyurek/Documents/code/ai/ai-maker-space/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [8]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What role does Abu Dhabi play in the developme...,[Code may be the best application The ethics o...,Abu Dhabi is involved in the development of la...,single_hop_specifc_query_synthesizer
1,What New York Times do?,[Based Development As a computer scientist and...,The New York Times launched a landmark lawsuit...,single_hop_specifc_query_synthesizer
2,Wht r LLMs?,[Simon Willison’s Weblog Subscribe Stuff we fi...,"LLMs, or Large Language Models, are the latest...",single_hop_specifc_query_synthesizer
3,Wut iz the impakt of Llama 2 on the developmin...,[easy to follow. The rest of the document incl...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,How has the accessibility of AI models evolved...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the accessibility of AI models has si...",multi_hop_abstract_query_synthesizer
5,How have advancements in AI model evaluation a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
6,How have advancements in AI model evaluation a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
7,How have advancements in AI model evaluation a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
8,How has the development of Claude Artifacts by...,[<1-hop>\n\nown output in a way that leads to ...,The development of Claude Artifacts by Anthrop...,multi_hop_specific_query_synthesizer
9,How has Meta contributed to the accessibility ...,[<1-hop>\n\nCode may be the best application T...,Meta has significantly contributed to the acce...,multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [10]:
dataset.upload()

KeyboardInterrupt: 

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [11]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Now that we have our data loaded, let's split it into chunks!

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

75

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [14]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

/Users/cantemizyurek/Documents/code/ai/ai-maker-space/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/cantemizyurek/Documents/code/ai/ai-maker-space/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


We can now add our documents to our vector store.

In [15]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [17]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [18]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [20]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [21]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [22]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [23]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [24]:
response["response"]

'LLM agents are useful in several ways, primarily through their ability to automate tasks and assist in problem-solving. There are two main perspectives on their utility:\n\n1. **Acting on Behalf of Users**: Some view LLM agents as digital assistants that can perform tasks for users, similar to a travel agent. They can handle requests and streamline processes, potentially saving users time and effort.\n\n2. **Problem Solving with Tools**: Others see LLMs as tools that can be integrated into workflows to solve problems iteratively. By accessing various tools and running tasks in a loop, they can contribute to more complex processes.\n\nDespite their potential benefits, there are significant challenges and skepticism regarding their reliability, particularly due to their tendency to hallucinate or generate incorrect information. This raises concerns about their effectiveness in making meaningful decisions without being able to distinguish truth from fiction.\n\nAdditionally, LLMs can be 

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [25]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [27]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What role does Abu Dhabi play in the developme...,[OpenAI are not the only game in town here. Go...,[Code may be the best application The ethics o...,Abu Dhabi plays a role in the development of l...,Abu Dhabi is involved in the development of la...,single_hop_specifc_query_synthesizer
1,What New York Times do?,"[Since then, almost every major LLM (and most ...",[Based Development As a computer scientist and...,The New York Times launched a landmark lawsuit...,The New York Times launched a landmark lawsuit...,single_hop_specifc_query_synthesizer
2,Wht r LLMs?,[So training an LLM still isn’t something a ho...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"LLMs, or Large Language Models, are advanced A...","LLMs, or Large Language Models, are the latest...",single_hop_specifc_query_synthesizer
3,Wut iz the impakt of Llama 2 on the developmin...,[Another common technique is to use larger mod...,[easy to follow. The rest of the document incl...,The impact of Llama 2 on the development of la...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,How has the accessibility of AI models evolved...,[Law is not ethics. Is it OK to train models o...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the accessibility of AI models has un...","In 2024, the accessibility of AI models has si...",multi_hop_abstract_query_synthesizer
5,How have advancements in AI model evaluation a...,"[If you can gather the right data, and afford ...",[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, advancements in AI model evaluation a...",Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
6,How have advancements in AI model evaluation a...,"[If you can gather the right data, and afford ...",[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, advancements in AI model evaluation a...",Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
7,How have advancements in AI model evaluation a...,"[If you can gather the right data, and afford ...",[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, advancements in AI model evaluation a...",Advancements in AI model evaluation and LLM pr...,multi_hop_abstract_query_synthesizer
8,How has the development of Claude Artifacts by...,[OpenAI are not the only game in town here. Go...,[<1-hop>\n\nown output in a way that leads to ...,The development of Claude Artifacts by Anthrop...,The development of Claude Artifacts by Anthrop...,multi_hop_specific_query_synthesizer
9,How has Meta contributed to the accessibility ...,[I wrote about how Large language models are h...,[<1-hop>\n\nCode may be the best application T...,Meta has significantly contributed to the acce...,Meta has significantly contributed to the acce...,multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [28]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [31]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))

Next up - we simply evaluate on our desired metrics!

In [32]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[13]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-2lA2EnUoLbqMmDswe8655Nmm on tokens per min (TPM): Limit 30000, Used 29335, Requested 2341. Please try again in 3.352s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[7]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-2lA2EnUoLbqMmDswe8655Nmm on tokens per min (TPM): Limit 30000, Used 29060, Requested 2156. Please try again in 2.432s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[24]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-2lA2EnUoLbqMmDswe8655Nmm on tokens per min (TPM): Limit 30000, Used 28800, Reques

{'context_recall': 0.7167, 'faithfulness': 0.9355, 'factual_correctness': 0.4227, 'answer_relevancy': 0.9497, 'context_entity_recall': 0.4635, 'noise_sensitivity_relevant': 0.3042}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [37]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

In [34]:
#!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [38]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [42]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [43]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [44]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents can be useful in certain contexts, particularly in writing code, as they demonstrate a strong capability in this area. The grammar rules of programming languages are less complex than natural languages, making it easier for LLMs to generate code effectively. However, their overall utility is questioned due to issues such as gullibility, where LLMs may struggle to distinguish truth from fiction. This skepticism is further amplified by the lack of real-world examples of LLM agents operating successfully in production environments, despite the excitement surrounding their potential. Thus, while LLM agents show promise, particularly in coding, concerns about their reliability and decision-making capabilities remain significant challenges that need to be addressed.'

In [45]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [47]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[65]: TimeoutError()
Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.7610, 'faithfulness': 0.8309, 'factual_correctness': 0.4275, 'answer_relevancy': 0.8605, 'context_entity_recall': 0.5109, 'noise_sensitivity_relevant': 0.3846}

#### ❓ Question: 

Which system performed better, on what metrics, and why?